In [2]:
import time
import json
import requests
import pandas as pd



def call_psipred_api(sequence: str):
    psipred = "http://bioinf.cs.ucl.ac.uk/psipred/api"
    submit_url = f"{psipred}/submission"
    fasta_sequence = f">query\n{sequence}"

    payload = {'input_data': fasta_sequence}
    data = {'job': 'psipred', 'submission_name': 'test','email': 'carrief0908@gmail.com'}
    r = requests.post(f"{submit_url}.json", data=data, files=payload)
    response_data = json.loads(r.text)
    print(response_data)
    uuid = response_data['UUID']

    retries = 0
    while retries < 30:
      result_uri = f"{submit_url}/{uuid}"
      r = requests.get(result_uri, headers={"Accept":"application/json"})
      result_data = json.loads(r.text)
      if "Complete" in result_data["state"]:
          data_path = result_data['submissions'][0]['results'][5]['data_path']
          response = requests.get(f"{psipred}{data_path}")
          if response.status_code != 200:
              raise Exception(f"Failed to get results: {response.text}")
          ss_sequence = ""
          for line in response.text.splitlines():
              if not line.startswith('#') and len(line.split()) > 2:
                  ss_sequence += line.split()[2]
          return ss_sequence
      else:
          retries += 1
          time.sleep(30)

    raise Exception("Timeout waiting for PSIPRED results")



split_data = pd.read_csv('../data/split.csv')
split_data = split_data[['Split Site', 'Sequence']]
split_data['Sequence'] = split_data['Sequence'].str.replace(' ', '', regex=False)

expanded_rows = split_data['Split Site'].str.split('/').explode()
expanded_data = pd.DataFrame({
    'Sequence': split_data.loc[expanded_rows.index, 'Sequence'].values,
    'Split Site': expanded_rows
})
expanded_data.reset_index(drop=True, inplace=True)

In [18]:
seqs = expanded_data['Sequence'].unique().tolist()[6:7]
Seqs = {}
for s in seqs:
    Seqs[s] = call_psipred_api(s)
    with open('../data/seq_2_second4.json', 'w') as f:
        json.dump(Seqs, f)

{'UUID': '1be48560-ada8-11ef-ae68-00163e100466', 'submission_name': 'test'}


Exception: Timeout waiting for PSIPRED results

In [ ]:
expanded_data['Secondary'] = expanded_data['Sequence'].map(Seqs)

In [ ]:
seqs[6]